# [ICTCOG] 4기 경북대 기본반

## Pandas (3)

### 추천 시스템

- Information Filtering
    - 검색:이용자가 능동적으로 정보를 찾음
    - 추천:사용자의 **이전 데이터의 패턴**를 바탕으로 필요한 **정보 제공**
    
> - 파레토 법칙: 소수의 20%집중(소품종 대량생산)
> - 롱테일 법칙: 다수의 80% (다품종 소량생산)    
    
> - Filter bubble
    
1. Collaborative filtering
 
2. Content-based filtering


- 유사성 계산
    - cosine similarity = $ cos(\theta)=\frac{AB}{||A|| ||B||} $
        - 클수록 가장 유사함(차이 적음)
        - 가장 유사한 유저들이 구매/사용 아이템 추천
        - K-nearest-neighbor 중에 구매하지 않은 목록 중 가장 큰 값
- recommended for edu &dev 1MB 참고: https://grouplens.org/datasets/movielens/ 

In [12]:
import numpy as np
import pandas as pd

In [8]:
links=pd.read_csv('ml-latest-small\links.csv')
links

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
9737,193581,5476944,432131.0
9738,193583,5914996,445030.0
9739,193585,6397426,479308.0
9740,193587,8391976,483455.0


In [9]:
movies=pd.read_csv('ml-latest-small\movies.csv')
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [15]:
ratings=pd.read_csv('ml-latest-small\\ratings.csv')
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [16]:
tags=pd.read_csv('ml-latest-small\\tags.csv')
tags

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200
...,...,...,...,...
3678,606,7382,for katie,1171234019
3679,606,7936,austere,1173392334
3680,610,3265,gun fu,1493843984
3681,610,3265,heroic bloodshed,1493843978


### 열 제거

In [36]:
r1= ratings.drop(['timestamp'],axis=1)  #timestamp 제거
r1

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
100831,610,166534,4.0
100832,610,168248,5.0
100833,610,168250,5.0
100834,610,168252,5.0


In [37]:
r1=ratings[ratings.columns[:3]]
r1

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
100831,610,166534,4.0
100832,610,168248,5.0
100833,610,168250,5.0
100834,610,168252,5.0


In [40]:
r1.set_index('userId')

,movieId,rating
userId,,
1,1,4.0
1,3,4.0
1,6,4.0
1,47,5.0
1,50,5.0
...,...,...
610,166534,4.0
610,168248,5.0
610,168250,5.0


In [47]:
r1.set_index(['userId','movieId'])#SCI 

rating
userId movieId        
1      1           4.0
       3           4.0
       6           4.0
       47          5.0
       50          5.0
...                ...
610    166534      4.0
       168248      5.0
       168250      5.0
       168252      5.0
       170875      3.0

[100836 rows x 1 columns]

추천시스템 만들기위한 핵심 형태


||movie id||
|-|-|-|
|user ID| rating|


In [51]:
# 값 없는 NaN ->0
r2=r1.set_index(['userId','movieId']).unstack()#SCI 
r2

rating                                                                 \
movieId 1      2      3      4      5      6      7      8      9      10       
userId                                                                          
1          4.0    NaN    4.0    NaN    NaN    4.0    NaN    NaN    NaN    NaN   
2          NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
3          NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
4          NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
5          4.0    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
...        ...    ...    ...    ...    ...    ...    ...    ...    ...    ...   
606        2.5    NaN    NaN    NaN    NaN    NaN    2.5    NaN    NaN    NaN   
607        4.0    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
608        2.5    2.0    2.0    NaN    NaN    NaN    NaN    NaN    NaN    4.0   
609        3.0    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    4.0   
610        5.0    NaN    NaN    NaN    NaN    5.0    NaN    NaN    NaN    NaN   

         ...                                                                 \
movieId  ... 193565 193567 193571 193573 193579 193581 193583 193585 193587   
userId   ...                                                                  
1        ...    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
2        ...    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
3        ...    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
4        ...    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
5        ...    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
...      ...    ...    ...    ...    ...    ...    ...    ...    ...    ...   
606      ...    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
607      ...    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
608      ...    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
609      ...    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
610      ...    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   

                
movieId 193609  
userId          
1          NaN  
2          NaN  
3          NaN  
4          NaN  
5          NaN  
...        ...  
606        NaN  
607        NaN  
608        NaN  
609        NaN  
610        NaN  

[610 rows x 9724 columns]

### 결츨치 처리
- 제거 Drop
    - drop, drop_duplicats, droplevel, dropna
- 대치 fill
    - fillna

In [54]:
r3=r2.fillna(0) #결측치 0으로 채움
r3

rating                                                                 \
movieId 1      2      3      4      5      6      7      8      9      10       
userId                                                                          
1          4.0    0.0    4.0    0.0    0.0    4.0    0.0    0.0    0.0    0.0   
2          0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
3          0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
4          0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
5          4.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
...        ...    ...    ...    ...    ...    ...    ...    ...    ...    ...   
606        2.5    0.0    0.0    0.0    0.0    0.0    2.5    0.0    0.0    0.0   
607        4.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
608        2.5    2.0    2.0    0.0    0.0    0.0    0.0    0.0    0.0    4.0   
609        3.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    4.0   
610        5.0    0.0    0.0    0.0    0.0    5.0    0.0    0.0    0.0    0.0   

         ...                                                                 \
movieId  ... 193565 193567 193571 193573 193579 193581 193583 193585 193587   
userId   ...                                                                  
1        ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
2        ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
3        ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
4        ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
5        ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
...      ...    ...    ...    ...    ...    ...    ...    ...    ...    ...   
606      ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
607      ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
608      ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
609      ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
610      ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   

                
movieId 193609  
userId          
1          0.0  
2          0.0  
3          0.0  
4          0.0  
5          0.0  
...        ...  
606        0.0  
607        0.0  
608        0.0  
609        0.0  
610        0.0  

[610 rows x 9724 columns]

#### pivot
- **하나의 cell 에 1개의 값**만 가능

In [69]:
r1.pivot(index='userId', columns='movieId' ,values='rating').fillna(0)

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
607,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
608,2.5,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [1]:
import seaborn as sns
tips= sns.load_dataset('tips')

In [65]:
tips.pivot_table('tip','sex','day',aggfunc='mean')

day,Thur,Fri,Sat,Sun
sex,,,,
Male,2.980333,2.693000,3.083898,3.220345
Female,2.575625,2.781111,2.801786,3.367222


### 상관관계
- 각 column 간의 상관관계( -1~ +1 )

In [76]:
r1.corr() 

,userId,movieId,rating
userId,1.000000,0.006773,-0.049348
movieId,0.006773,1.000000,-0.004061
rating,-0.049348,-0.004061,1.000000


In [ ]:
t3.corrwith() #2개 이상

In [78]:
r3.T #전치

userId          1    2    3    4    5    6    7    8    9    10   ...  601  \
       movieId                                                    ...        
rating 1        4.0  0.0  0.0  0.0  4.0  0.0  4.5  0.0  0.0  0.0  ...  4.0   
       2        0.0  0.0  0.0  0.0  0.0  4.0  0.0  4.0  0.0  0.0  ...  0.0   
       3        4.0  0.0  0.0  0.0  0.0  5.0  0.0  0.0  0.0  0.0  ...  0.0   
       4        0.0  0.0  0.0  0.0  0.0  3.0  0.0  0.0  0.0  0.0  ...  0.0   
       5        0.0  0.0  0.0  0.0  0.0  5.0  0.0  0.0  0.0  0.0  ...  0.0   
...             ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
       193581   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
       193583   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
       193585   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
       193587   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
       193609   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   

userId          602  603  604  605  606  607  608  609  610  
       movieId                                               
rating 1        0.0  4.0  3.0  4.0  2.5  4.0  2.5  3.0  5.0  
       2        4.0  0.0  5.0  3.5  0.0  0.0  2.0  0.0  0.0  
       3        0.0  0.0  0.0  0.0  0.0  0.0  2.0  0.0  0.0  
       4        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
       5        0.0  0.0  3.0  0.0  0.0  0.0  0.0  0.0  0.0  
...             ...  ...  ...  ...  ...  ...  ...  ...  ...  
       193581   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
       193583   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
       193585   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
       193587   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
       193609   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[9724 rows x 610 columns]

In [80]:
r3_corr=r3.T.corr() #유저간의 유사도 비교

In [84]:
r3_corr[[307]] #유저 307

userId,307
userId,
1,0.239943
2,0.045427
3,0.001814
4,0.145341
5,0.130645
...,...
606,0.209710
607,0.192797
608,0.366609


유저 307과 가장 양의 상관관계가 높은사람(유사한 사람)

In [92]:
r3_corr.loc[307].sort_values(ascending=False)

userId
307    1.000000
480    0.428839
226    0.417297
274    0.402623
64     0.387990
         ...   
3      0.001814
578    0.001716
545   -0.000498
175   -0.000588
138   -0.005768
Name: 307, Length: 610, dtype: float64

- array programming 으로 간단

In [96]:
r3_corr.loc[307].nlargest(5)[1:] #4명

userId
480    0.428839
226    0.417297
274    0.402623
64     0.387990
Name: 307, dtype: float64

In [95]:
r3_corr.loc[307].sort_values(ascending=False)[1:5]

userId
480    0.428839
226    0.417297
274    0.402623
64     0.387990
Name: 307, dtype: float64

함수로 만들기

In [97]:
def best_similarity(userID, n_largest=5):
    ''' userID 입력시 n명의 가장 유사한 유저 ID와 상관관계 리턴'''
    return r3_corr.loc[userID].nlargest(n_largest)[1:]   

In [98]:
best_similarity(307)

userId
480    0.428839
226    0.417297
274    0.402623
64     0.387990
Name: 307, dtype: float64

In [100]:
best_similarity(11,6)

userId
486    0.447245
176    0.378167
340    0.371425
126    0.364705
379    0.353583
Name: 11, dtype: float64

유저가 본 영화

In [117]:
ratings[ratings.userId==307] #영화 975개봄

,userId,movieId,rating,timestamp
46733,307,1,4.0,1186160893
46734,307,2,2.5,1186161482
46735,307,3,3.5,1186161652
46736,307,10,2.5,1186161010
46737,307,16,4.5,1186087665
...,...,...,...,...
47703,307,66097,4.5,1242822491
47704,307,67408,4.0,1242822456
47705,307,68952,4.5,1244814311
47706,307,68954,5.0,1243884778


In [118]:
ratings[ratings.userId==307]['movieId']

46733        1
46734        2
46735        3
46736       10
46737       16
         ...  
47703    66097
47704    67408
47705    68952
47706    68954
47707    69122
Name: movieId, Length: 975, dtype: int64

In [122]:
ratings[ratings.userId==480]['movieId']

76227        1
76228        2
76229        3
76230        6
76231       10
         ...  
77058    49651
77059    51255
77060    51662
77061    52973
77062    53000
Name: movieId, Length: 836, dtype: int64

유저 480만 본 영화

In [128]:
exclusive=set(ratings[ratings.userId==480]['movieId'])-set(ratings[ratings.userId==307]['movieId'])
exclusive #442 게

{6,
 34,
 60,
 69,
 95,
 160,
 161,
 163,
 168,
 172,
 173,
 196,
 246,
 249,
 261,
 282,
 293,
 300,
 316,
 317,
 348,
 349,
 370,
 380,
 382,
 420,
 432,
 440,
 442,
 474,
 481,
 485,
 508,
 552,
 653,
 741,
 802,
 832,
 954,
 991,
 1060,
 1078,
 1079,
 1080,
 1090,
 1103,
 1127,
 1179,
 1183,
 1200,
 1203,
 1208,
 1225,
 1235,
 1242,
 1256,
 1266,
 1274,
 1275,
 1278,
 1282,
 1302,
 1304,
 1320,
 1321,
 1370,
 1377,
 1378,
 1394,
 1396,
 1408,
 1449,
 1464,
 1466,
 1527,
 1573,
 1610,
 1625,
 1641,
 1653,
 1676,
 1690,
 1693,
 1722,
 1747,
 1876,
 1921,
 1954,
 1957,
 1962,
 2001,
 2006,
 2028,
 2080,
 2085,
 2094,
 2167,
 2194,
 2231,
 2273,
 2318,
 2324,
 2353,
 2355,
 2396,
 2455,
 2539,
 2599,
 2617,
 2664,
 2671,
 2692,
 2701,
 2722,
 2728,
 2763,
 2770,
 2788,
 2863,
 2872,
 2916,
 2946,
 2947,
 2948,
 2973,
 3000,
 3020,
 3037,
 3100,
 3148,
 3168,
 3176,
 3178,
 3189,
 3256,
 3285,
 3300,
 3301,
 3317,
 3355,
 3360,
 3362,
 3386,
 3418,
 3471,
 3481,
 3484,
 3527,
 3555,
 35

### contents based filtering
-  많이 본 영화 추천

In [142]:
ratings.groupby('movieId').count().sort_values('userId',ascending=False)[10:]

,userId,rating,timestamp
movieId,,,
2959,218,218,218
1,215,215,215
1196,211,211,211
50,204,204,204
2858,204,204,204
...,...,...,...
4093,1,1,1
4089,1,1,1
58351,1,1,1


가장 많이 본 영화 10개 중 평점 높은 5개 

In [143]:
ratings.groupby('movieId').count().sort_values('userId',ascending=False)[10:]['rating']

movieId
2959      218
1         215
1196      211
50        204
2858      204
         ... 
4093        1
4089        1
58351       1
4083        1
193609      1
Name: rating, Length: 9714, dtype: int64

In [157]:
t=ratings.groupby('movieId').count().sort_values('userId',ascending=False)[:10]['rating'].index

In [158]:
ratings[ratings.movieId.isin(t)]

,userId,movieId,rating,timestamp
7,1,110,4.0,964982176
15,1,260,5.0,964981680
16,1,296,3.0,964982967
20,1,356,4.0,964980962
26,1,480,4.0,964982346
...,...,...,...,...
99569,610,480,5.0,1479545821
99571,610,527,3.5,1479545826
99573,610,589,5.0,1479542983
99575,610,593,4.5,1479545819


In [159]:
ratings[ratings.movieId.isin(t)].groupby('movieId')['rating'].mean().nlargest(5)

movieId
318     4.429022
260     4.231076
527     4.225000
296     4.197068
2571    4.192446
Name: rating, dtype: float64

쇼생크 탈출 영화는 가장 많이 본 영화 중 평균 평점 제일 높음

In [160]:
movies[movies.movieId==318] #쇼생크 탈출

,movieId,title,genres
277,318,"Shawshank Redemption, The (1994)",Crime|Drama


In [162]:
best_similarity(307,4)

userId
480    0.428839
226    0.417297
274    0.402623
Name: 307, dtype: float64

유저 226의 평점 낮은 최악의 상위 10개 영화

In [167]:
ratings[ratings.userId==226].sort_values('rating')[:10]

,userId,movieId,rating,timestamp
33500,226,32666,1.0,1201155937
33152,226,1088,1.0,1096420160
33354,226,4229,1.0,1095695395
33196,226,1544,1.5,1160003791
33488,226,8810,1.5,1095663572
...,...,...,...,...
33415,226,5650,5.0,1095713547
33207,226,1732,5.0,1095663026
33220,226,2005,5.0,1095663312
33326,226,3671,5.0,1095663341


In [178]:
movies[movies.movieId== 32666]

,movieId,title,genres
5860,32666,National Lampoon's Lady Killers (National Lamp...,Comedy


In [179]:
movies[movies.movieId== 1088]

,movieId,title,genres
827,1088,Dirty Dancing (1987),Drama|Musical|Romance


유저 226이 본 최악의 영화 6개 중 유저 307 이 안 본 영화 추천

In [186]:
ratings[ratings.userId==226][['rating','movieId']]

,rating,movieId
33044,3.5,1
33045,3.0,2
33046,3.5,3
33047,4.0,10
33048,4.5,16
...,...,...
33546,3.5,56367
33547,4.0,56757
33548,3.5,56805
33549,2.5,59615


In [187]:
ratings[ratings.userId==226][['rating','movieId']].sort_values('rating').head(6)
#['movieId'].values


,rating,movieId
33323,0.5,3593
33138,0.5,888
33452,0.5,6880
33484,0.5,8666
33355,0.5,4233
33397,0.5,5323


In [188]:
ratings[ratings.userId==226][['rating','movieId']].sort_values('rating').head(6)['movieId']

33323    3593
33138     888
33452    6880
33484    8666
33355    4233
33397    5323
Name: movieId, dtype: int64

In [183]:
worst=ratings[ratings.userId==226][['rating','movieId']].sort_values('rating').head(6)['movieId'].values
worst

array([3593,  888, 6880, 8666, 4233, 5323], dtype=int64)

In [185]:
set(worst)-set(ratings[ratings.userId==307]['movieId'].values) #제미없는 6개 추천

{888, 3593, 4233, 5323, 6880, 8666}

- 최고 영화 50개 중 유저 307이 보지 않은 영화 추천

In [199]:
best=ratings[ratings.userId==226][['rating','movieId']].sort_values('rating').tail(50)['movieId'].values
best

array([ 1197,  2984,  1527,  7361,  3052,  1291,  1288,  1276,  2804,
        1262,  6711,  8376,  3275,  2329,  2423,  2268,  1257,  6539,
        3421,  8784,  2161,  3360,  5989, 55765,  6188, 55820,    47,
        4571,   170,   180,   216,   104, 46578,   231,  1196,  1210,
        1136,  3653,  1073,  1222,   223,  5110,   671,  3253,  3033,
        5650,  1732,  2005,  3671,  2502], dtype=int64)

In [208]:
[*set(best)-set(ratings[ratings.userId==307]['movieId'].values)]

[3360, 3653, 2984, 170, 6188, 1262, 1276, 1527, 8376, 2268, 671]

In [222]:
movies.loc[[*set(best)-set(ratings[ratings.userId==307]['movieId'].values)]]

,movieId,title,genres
3360,4564,Always (1989),Drama|Fantasy|Romance
3653,5021,Murder by Death (1976),Comedy|Crime|Mystery|Thriller
2984,3997,Dungeons & Dragons (2000),Action|Adventure|Comedy|Fantasy
170,202,Total Eclipse (1995),Drama|Romance
6188,44937,"Child, The (L'enfant) (2005)",Crime|Drama
1262,1677,Critical Care (1997),Comedy|Drama
1276,1693,Amistad (1997),Drama|Mystery
1527,2059,"Parent Trap, The (1998)",Children|Comedy|Romance
8376,109487,Interstellar (2014),Sci-Fi|IMAX
2268,3010,Rosetta (1999),Drama


In [254]:
worst10=ratings.groupby('movieId').mean().sort_values('rating')[:10]
worst10

,userId,rating,timestamp
movieId,,,
26696,599.0,0.5,1.498503e+09
3604,89.0,0.5,1.520409e+09
7312,88.0,0.5,1.331421e+09
145724,599.0,0.5,1.519115e+09
76030,599.0,0.5,1.498503e+09
145951,610.0,0.5,1.493849e+09
85334,358.0,0.5,1.339546e+09
129250,448.0,0.5,1.447869e+09
102025,448.0,0.5,1.492718e+09


In [255]:
best10=ratings.groupby('movieId').mean().sort_values('rating',ascending=False)[:10]
best10

,userId,rating,timestamp
movieId,,,
88448,483.0,5.0,1.315438e+09
100556,462.0,5.0,1.456151e+09
143031,89.0,5.0,1.520409e+09
143511,105.0,5.0,1.526207e+09
143559,89.0,5.0,1.520410e+09
6201,474.0,5.0,1.100120e+09
102217,63.0,5.0,1.443200e+09
102084,380.0,5.0,1.493422e+09
6192,182.0,5.0,1.063275e+09


In [274]:
movies.loc[6192]

movieId                               45028
title      Prairie Home Companion, A (2006)
genres                 Comedy|Drama|Musical
Name: 6192, dtype: object

- 307과 유사한 3명이 공통적으로 본 영화중에서 평점 4.5 이상, 가장 최신 영화 추천


In [287]:
best_similarity(307,4).

Int64Index([480, 226, 274], dtype='int64', name='userId')

In [300]:
a=ratings[ratings.userId == 480].movieId.values
b=ratings[ratings.userId == 226].movieId.values
c=ratings[ratings.userId == 274].movieId.values

In [301]:
ai=pd.Index(a)
bi=pd.Index(b)
ci=pd.Index(c)

In [303]:
ai.intersection(bi).intersection(ci) #3이 공통적으로 본 영화

Int64Index([    1,     2,    10,    16,    19,    32,    47,    60,    69,
              104,
            ...
            45447, 45722, 46335, 46578, 46970, 48516, 48780, 51255, 52973,
            53000],
           dtype='int64', length=235)

In [309]:
#235개의 공통적 영화
data=movies.set_index('movieId').loc[ai.intersection(bi).intersection(ci) ]
data

,title,genres
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
10,GoldenEye (1995),Action|Adventure|Thriller
16,Casino (1995),Crime|Drama
19,Ace Ventura: When Nature Calls (1995),Comedy
...,...,...
48516,"Departed, The (2006)",Crime|Drama|Thriller
48780,"Prestige, The (2006)",Drama|Mystery|Sci-Fi|Thriller
51255,Hot Fuzz (2007),Action|Comedy|Crime|Mystery
52973,Knocked Up (2007),Comedy|Drama|Romance


정규식 이용해서 패턴 뽑아 최신영화 구분

In [308]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


### str
- series 에서 str 기존에 정의되어있던 function으로 정규식이용 추출
### split
- pat : 정규식 패턴
    - [0-9] 숫자
    


In [313]:
data.title.str.split('(')

1                             [Toy Story , 1995)]
2                               [Jumanji , 1995)]
10                            [GoldenEye , 1995)]
16                               [Casino , 1995)]
19       [Ace Ventura: When Nature Calls , 1995)]
                           ...                   
48516                     [Departed, The , 2006)]
48780                     [Prestige, The , 2006)]
51255                          [Hot Fuzz , 2007)]
52973                        [Knocked Up , 2007)]
53000                    [28 Weeks Later , 2007)]
Name: title, Length: 235, dtype: object

In [319]:
x=data.title.str.split('(',expand=True) # None으로 됨
x

,0,1,2
1,Toy Story,1995),None
2,Jumanji,1995),None
10,GoldenEye,1995),None
16,Casino,1995),None
19,Ace Ventura: When Nature Calls,1995),None
...,...,...,...
48516,"Departed, The",2006),None
48780,"Prestige, The",2006),None
51255,Hot Fuzz,2007),None
52973,Knocked Up,2007),None


In [320]:
x[x[2]==None]

,0,1,2


In [321]:
x[2].isna()

1        True
2        True
10       True
16       True
19       True
         ... 
48516    True
48780    True
51255    True
52973    True
53000    True
Name: 2, Length: 235, dtype: bool

In [322]:
x[2].isnull()

1        True
2        True
10       True
16       True
19       True
         ... 
48516    True
48780    True
51255    True
52973    True
53000    True
Name: 2, Length: 235, dtype: bool

In [323]:
x[ x[2].isna()]

,0,1,2
32,Twelve Monkeys,a.k.a. 12 Monkeys),1995)
47,Seven,a.k.a. Se7en),1995)
231,Dumb & Dumber,Dumb and Dumber),1994)
780,Independence Day,a.k.a. ID4),1996)
1198,Raiders of the Lost Ark,Indiana Jones and the Raiders of the Lost Ark),1981)
1580,Men in Black,a.k.a. MIB),1997)
2085,101 Dalmatians,One Hundred and One Dalmatians),1961)
2716,Ghostbusters,a.k.a. Ghost Busters),1984)
3703,"Road Warrior, The",Mad Max 2),1981)
4896,Harry Potter and the Sorcerer's Stone,a.k.a. Harry Potter and the Philosopher's Stone),2001)


In [332]:
x=data.title.str.split('[0-9]',n=1,expand=True) # None으로 됨
x

,0,1
1,Toy Story (,995)
2,Jumanji (,995)
10,GoldenEye (,995)
16,Casino (,995)
19,Ace Ventura: When Nature Calls (,995)
...,...,...
48516,"Departed, The (",006)
48780,"Prestige, The (",006)
51255,Hot Fuzz (,007)
52973,Knocked Up (,007)


In [325]:
x=data.title.str.split('(',n=1,expand=True) #2등분
x

,0,1
1,Toy Story,1995)
2,Jumanji,1995)
10,GoldenEye,1995)
16,Casino,1995)
19,Ace Ventura: When Nature Calls,1995)
...,...,...
48516,"Departed, The",2006)
48780,"Prestige, The",2006)
51255,Hot Fuzz,2007)
52973,Knocked Up,2007)


In [333]:
data.title

1                             Toy Story (1995)
2                               Jumanji (1995)
10                            GoldenEye (1995)
16                               Casino (1995)
19       Ace Ventura: When Nature Calls (1995)
                         ...                  
48516                     Departed, The (2006)
48780                     Prestige, The (2006)
51255                          Hot Fuzz (2007)
52973                        Knocked Up (2007)
53000                    28 Weeks Later (2007)
Name: title, Length: 235, dtype: object

- map은 series에서만 사용 가능
- 특정 위치의 데이터 추출

In [336]:
data.title.map(lambda x: x[-5:-1]) 

1        1995
2        1995
10       1995
16       1995
19       1995
         ... 
48516    2006
48780    2006
51255    2007
52973    2007
53000    2007
Name: title, Length: 235, dtype: object

In [337]:
data['year']= data.title.map(lambda x: x[-5:-1]) 

In [338]:
data

,title,genres,year
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
2,Jumanji (1995),Adventure|Children|Fantasy,1995
10,GoldenEye (1995),Action|Adventure|Thriller,1995
16,Casino (1995),Crime|Drama,1995
19,Ace Ventura: When Nature Calls (1995),Comedy,1995
...,...,...,...
48516,"Departed, The (2006)",Crime|Drama|Thriller,2006
48780,"Prestige, The (2006)",Drama|Mystery|Sci-Fi|Thriller,2006
51255,Hot Fuzz (2007),Action|Comedy|Crime|Mystery,2007
52973,Knocked Up (2007),Comedy|Drama|Romance,2007


In [341]:
data.title= data.title.map(lambda x: x[:-6])

In [342]:
data

,title,genres,year
1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
2,Jumanji,Adventure|Children|Fantasy,1995
10,GoldenEye,Action|Adventure|Thriller,1995
16,Casino,Crime|Drama,1995
19,Ace Ventura: When Nature Calls,Comedy,1995
...,...,...,...
48516,"Departed, The",Crime|Drama|Thriller,2006
48780,"Prestige, The",Drama|Mystery|Sci-Fi|Thriller,2006
51255,Hot Fuzz,Action|Comedy|Crime|Mystery,2007
52973,Knocked Up,Comedy|Drama|Romance,2007


- 하나의 셀에 중복되기 때문에 각 장르에 따라 계산하여 하나의 값으로 만듦

In [348]:
def genres(x):
    temp=0 #accumulation pattern
    if 'Action' in x:
        temp +=2**0
    if 'Adventure' in x:
        temp +=2**1
    if 'Animation' in x:
        temp +=2**2
    if 'Comedy' in x:
        temp +=2**4     
    return temp

- map 이용하여 여러 값 처리

In [350]:
data.genres.map(genres)

1        22
2         2
10        3
16        0
19       16
         ..
48516     0
48780     0
51255    17
52973    16
53000     0
Name: genres, Length: 235, dtype: int64

In [352]:
data.genres =data.genres.map(genres)
data 

,title,genres,year
1,Toy Story,22,1995
2,Jumanji,2,1995
10,GoldenEye,3,1995
16,Casino,0,1995
19,Ace Ventura: When Nature Calls,16,1995
...,...,...,...
48516,"Departed, The",0,2006
48780,"Prestige, The",0,2006
51255,Hot Fuzz,17,2007
52973,Knocked Up,16,2007
